# This notebook is more for show the what and why of the code. The production code is in the `/scr` folder

### You'll want to be in the `'../International-Maritime-Piracy/notebooks'` folder to get started

In [3]:
import os
import re

import pandas as pd

In [4]:
data_directory = os.path.join('../data','RAW_ASAM.csv')

df = pd.read_csv(data_directory, skipinitialspace = True)


df.head()

,reference,date,position,navArea,subreg,hostility,victim,description,Unnamed: 8
0,2022-127,2022-04-28,"13°43'00""N \n121°02'00""E",XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ...",NaN
1,2022-124,2022-04-27,"1°16'00""N \n104°17'00""E",XI,71,Boarding,Bulk carrier,"INDONESIA On 27 April, at 0226 local time, a r...",NaN
2,2022-121,2022-04-26,"0°04'00""S \n51°06'00""W",V,24,Boarding,Bulk carrier,"BRAZIL: On 26 April, at 0400 local time, three...",NaN
3,2022-128,2022-04-26,"22°13'00""N \n91°43'00""E",VIII,63,NaN,Singapore-flagged tanker STI MAGISTER,"BANGLADESH: On 26 April, at 0001 local time, a...",NaN
4,2022-122,2022-04-25,"59°19'00""N \n10°30'00""E",I,44,Protest,Hong Kong-flagged tanker UST LUGA,"NORWAY: On the morning of 25 April, a group of...",NaN


In [5]:
df.drop(['Unnamed: 8'], axis=1, inplace=True)

In [6]:
df

,reference,date,position,navArea,subreg,hostility,victim,description
0,2022-127,2022-04-28,"13°43'00""N \n121°02'00""E",XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ..."
1,2022-124,2022-04-27,"1°16'00""N \n104°17'00""E",XI,71,Boarding,Bulk carrier,"INDONESIA On 27 April, at 0226 local time, a r..."
2,2022-121,2022-04-26,"0°04'00""S \n51°06'00""W",V,24,Boarding,Bulk carrier,"BRAZIL: On 26 April, at 0400 local time, three..."
3,2022-128,2022-04-26,"22°13'00""N \n91°43'00""E",VIII,63,NaN,Singapore-flagged tanker STI MAGISTER,"BANGLADESH: On 26 April, at 0001 local time, a..."
4,2022-122,2022-04-25,"59°19'00""N \n10°30'00""E",I,44,Protest,Hong Kong-flagged tanker UST LUGA,"NORWAY: On the morning of 25 April, a group of..."
...,...,...,...,...,...,...,...,...
8614,1985-18,1980-01-11,"33°60'00""N \n33°60'00""E",III,56,PIRATES,INSTERGENERAL R. GUMUSPALA,"Instergeneral R. Gumuspala, a Turkish-flag gen..."
8615,1985-4,1979-10-10,"16°60'00""N \n112°00'00""E",XI,93,NaN,NaN,"SOUTH CHINA SEA, PARACEL ISLANDS. ..."
8616,1985-3,1979-02-24,"19°00'00""N \n108°00'00""E",XI,93,NaN,NaN,GULF OF TONKIN. ...
8617,1985-2,1978-05-16,"7°60'00""N \n119°60'00""E",XI,92,PIRATES,MERCHANT VESSEL,"SULU SEA: Lloyd's List of 15 May reports a 5,0..."


In [7]:
set(df['hostility'])

{'15 PIRATES',
 '24-APR-2002',
 '25 SPEEDBOATS',
 'ACTIVISTS',
 'ALGERIAN COASTAL PATROL',
 'ALLEGED POLISARIO FRONT',
 'AMERBER II',
 'ARMED PIRATES',
 'ARMED ROBBERS',
 'ARMED ROBBERY',
 'ASSAILANTS',
 'ATTACKER',
 'ATTACKERS',
 'ATTEMPTED BOARDING',
 'AUTHORITIES IN JACMEL, HAITI',
 'Algerian Patrol Vessel',
 'Approached by fishing boat with four armed men',
 'Approached by skiff',
 'Armed Pirates',
 'Armed boarders',
 'Armed men in a speedboat',
 'Armed persons boarding',
 'Armed persons in a skiff fired upon a tanker',
 'Armed robbers boarded vessel',
 'Armed robbers boarding',
 'Armed robebrs',
 'Assailants',
 'Attacked and boarded',
 'Attacked and fired upon',
 'Attacked three passenger boats',
 'Attempted Boarding',
 'Attempted boarding',
 'Attempted robbery',
 'BANDITS',
 'BLUE SPEEDBOAT',
 'BOARDED BY ARMED ROBBERS',
 'BOARDER',
 'BOARDERS',
 'BOARDING',
 'BOAT',
 'BURMESE PIRATES',
 'Bandits',
 'Boarded by five robbers',
 'Boarders',
 'Boarding',
 'Boarding and attempted rob

In [6]:
df['hostility'].nunique()

430

In [8]:
set(df['victim'])

{nan,
 'MSC CARMEN',
 'VERANO/CORUNA/DOAL UCKA',
 'BAHMAS',
 'General cargo ship',
 'M/V SEA WOLF',
 'General Cargo Ship',
 'SOUTH PACIFIC; PACIFICA 1',
 'UNKNOWN CRAFT',
 'PRODUCT TANKER LOULOU',
 'MERCHANT VESSEL',
 'AKIJ PEARL',
 'Oil Platform',
 'Cameroon-flagged cargo vessel STEVIA',
 'AQUITANIA',
 'BAND AID STAR',
 'Liberia-flagged bulk carrier SMARTA',
 'BULK CARRIER WESTGATE',
 'Singapore-flagged bulk carrier LAMPARD',
 'Liberia-flagged bulk carrier TRUDY',
 'BARGE UNDER TOW',
 'Refrigerated Cargo Ship',
 'TUG FOKU MARU NO. 8',
 'URUGUAY EXPRESS',
 'CHEMICAL TANKER GANDHI',
 'Passenger Boat',
 'AUSTRALIAN STAR',
 'STRONGHAND',
 'CARRIER',
 'M/V SO REVALU',
 'Tankers',
 'Barge under tow',
 'LPG CARRIER',
 'container ship APL LOS ANGELES',
 'INABUKWA',
 'Panama-flagged chemical tanker AEGEAN Il',
 'Tanker Crewmen',
 'OCEAN STAR',
 'SAILBOAT',
 'M/V TORBEN SPIRIT/M/V ESSBERGER PIONEER',
 'HARRISBURG',
 'LIVESTOCK CARRIER',
 'Barge being towed by tug',
 'Civilian',
 'Tanker at anch

In [9]:
df['victim'].nunique()

1588

In [2]:
df.info()

NameError: name 'df' is not defined

### Split postion into Lat/Lon in prep for conversion to decimal degrees

In [ ]:
df = df.join(df['position'].str.split(expand=True)).rename(columns={0:'latitude', 1:'longitude'})
df.drop('position', axis=1, inplace=True)

In [ ]:
df.head(1)

In [ ]:
def dms2dd(field):
    # example: s = """0°51'56.29"S"""
    degrees, minutes, seconds, direction = re.split('[°\'"]+', field)
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd

In [ ]:
df['latitude'] = df['latitude'].apply(dms2dd).round(6)
df['longitude'] = df['longitude'].apply(dms2dd).round(6)
df.head(4)

### What's up with the 'hostility' column?

In [ ]:
df['hostility'].value_counts()[100:150]

### Under 'hostility' - how many different terms are there?

In [ ]:
df['hostility'].nunique()

### Under 'victim' - how many different terms are there?

In [ ]:
df['victim'].nunique()

In [ ]:
df['victim'].value_counts()[900:950]

### Convert datetime to just the year for a slider

In [ ]:
df['date2'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df.head(1)

In [ ]:
df['year'] = pd.DatetimeIndex(df['date2']).year
df["date"] = pd.to_datetime(df["date2"])

df = df[['reference', 'latitude', 'longitude', 'date', 'year', 'description', 'hostility', 'victim', 'navArea', 'subreg' ]]

# sanity check
df

### Right, the NaNs would mess things up. Let's convert them to something that can be parsed later

In [ ]:
df.fillna("No information", inplace=True)
df.isnull().sum().sum()
df

In [ ]:
os.makedirs('../data/in_progress', exist_ok=True)  

df.to_csv('../data/in_progress/v1_lat_long.csv') 

# Future work

- [x] Remove the excess spaces in the description column.


- [X] Fix the 'reference' column values
    - [x] contact NGA and see if this would be of interest (**working**)
    

- [ ] Regularize the 'hostility' columns values - easy first pass would be to trust the data ruthlessly pare down the columns


- [ ] Regularize the 'victim' columns values  - easy first pass would be to trust the data ruthlessly pare down the columns


- [ ] NLP model to set the hostility and possibly victim columns




# <center>Remove the excess spaces in the 'description' column</center>


The excel file for the description has a lot a weird spacing. Does that show here?

* *Why, yes it does.* But it only shows if you have a wide column width

Also, another plug for `skipinitialspace = True` flag without it you need to add `df.hostility = df.hostility.str.lstrip()` 

In [ ]:
pd.set_option('display.max_colwidth', None)
df['description'][df["reference"]=='1991-21']

In [ ]:
df

In [ ]:
df.replace('\s+', ' ',regex=True,inplace=True)

In [ ]:
# OK, that seems to have done it.
#df

In [ ]:
df.groupby(pd.Grouper(key="date")).mean()
# gp = gp.divide(gp.sum(axis=1), axis=0).multiply(100)
df

# <center>Fix the 'reference' number column values</center>

In [ ]:
df['reference'] = (df['date'].dt.year.astype(str) + '-' + (df.sort_values('date')
                    .groupby(df['date'].dt.year).cumcount()+1).astype(str))

In [ ]:
df

# <center>Regularize the 'hostility' columns values</center>

This is tricky and I think and NLP approach will be the better overall but in terms of getting something out the door, I think this is fine.

In [ ]:
def hostilities(df):
    h = df.hostility.unique()
    h.sort()
    print(h)

def body_count(df):
    return df.hostility.nunique()

In [ ]:
first_cut = body_count(df)
first_cut

In [ ]:
hostilities(df)

### Nobody can spell ~~SIUSPICIOUS~~, ~~suspicous~~, ~~SUS\[ICIOUS~~, ~~SUSPIPICIOUS~~ suspicious

In [ ]:
#df[df["hostility"].str.contains(r"^Su(s|c)(p|])", regex=True, case=False)]

df.hostility = df.hostility.str.lstrip()
df['hostility'] = df['hostility'].str.replace(r'(?i)^Su[sc][][p][ie].*', 'Suspicious Approach', regex=True)

In [ ]:
second_cut = body_count(df)
print(f" This took us from {first_cut} to {second_cut}")

### Protestors and Stoaways seem simple

In [ ]:
df['hostility'] = df['hostility'].str.replace(r'(?i)Protest.*', 'Protest', regex=True)
df['hostility'] = df['hostility'].str.replace(r'(?i)Stow.*', 'Stowaway', regex=True)

In [ ]:
third_cut = body_count(df)
print(f" This took us from {second_cut} to {third_cut}")

### A whole lot of robberies to take care of

Note:
Creating an extra column:

```
df['Robbery?'] = df["hostility"].str.contains("robbe*", case=False, na=False).astype(int)
``` 
followed by:
```
df.loc[df["hostility"].str.contains("robbe", case=False, na=False)] = "OMG! A ROBBERY!"
```
was a good way to make sure to step though things.

In [ ]:
88888888 ----------->>>  stopping point  <<<------------- 8888888

In [ ]:
df.loc[df["hostility"].str.contains("robbe", case=False, na=False)] = "Robbery"

Oddly enough there is no 'Piracy' category in a database on pirates. The categories are:

- Attempted Boarding – Close approach or hull-to-hull contact with report that boarding
paraphernalia were employed or visible in the approaching boat.
- Blocking – Hampering safe navigation, docking, or undocking of a vessel as a means of protest.
- Boarding – Unauthorized embarkation of a vessel by persons not part of its complement without
successfully taking control of the vessel.
- Fired Upon – Weapons discharged at or toward a vessel.
- Hijacking – Unauthorized seizure and retention of a vessel by persons not part of its complement.
- Kidnapping – Unauthorized forcible removal of persons belonging to the vessel from it.
- Hijacking/Kidnapping Combination – Unauthorized seizure and retention of a vessel by persons not
- Robbery – Theft from a vessel or from persons aboard the vessel.
- Suspicious Approach 

In [ ]:
df.loc[df["hostility"].str.contains("Pirate*", case=False, na=False)] = "OOOOHHHH Pirates!"

In [ ]:
df['hostility'].nunique()

In [ ]:
os.makedirs('../data/in_progress', exist_ok=True)  

df.to_csv('../data/in_progress/v1_lat_long.csv') 